# Разметка корпуса и работа с ним

На этом семинаре у нас две задачи:

1. Научиться делать морфологическую и синтаксическую разметку корпуса.
2. Научиться работать с ним: извлекать и обобщать информацию.

[Ссылка](https://colab.research.google.com/drive/1yLZhkU6_aaSQqnMNzhvOFSohpmqVqImV?usp=sharing) на тетрадку в Google Colab.

In [38]:
%%capture
!pip install stanza
!pip install pandas

In [39]:
#Выполнение запросов в интернет — используем, чтобы скачать файл
import requests

#Работа с таблицами — понадобится, чтобы хранить наш корпус
import pandas as pd

#Выполнение быстрых математических вычислений — пригодится, потому что иначе
#корпус с большим количеством текстов может долго обрабатываться
import numpy as np

#Разметка морфологии и синтаксиса
import stanza

## Разметка

Будем использовать Stanza — см. [предыдущий конспект](https://github.com/alekseyst/text_analysis_2028/blob/main/Practical_3/Practical_3_Annotation.ipynb).

In [40]:
#Скачиваем обученную для русского языка модель
stanza.download("ru")

#Загружаем модель
nlp = stanza.Pipeline(lang="ru", processors="tokenize, pos, lemma, depparse")

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: ru (Russian) ...
INFO:stanza:File exists: /root/stanza_resources/ru/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |
| depparse  | syntagrus_charlm   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


## Разметка длинного текста

Как нам хранить информацию о длинном тексте? Наверное, будет не удобно размечать его каждый раз, когда он нам потребуется. Да и для поиска объект, который хранит Stanza, не самый удобный.

Возможный вариант — преобразовать размеченный текст в таблицу, где в каждой строчке содержится информация об одном слове. Таким образом можно будет сохранить текст в файл — и в следующий раз не парсить текст заново, а только обратиться к уже размеченному файлу. Плюс для поиска и анализа даных в таблице есть удобный инструментарий.

Для работы с таблицами будем использовать библиотеку Pandas.

In [95]:
#Скачиваем текстовый файл с книгой

response = requests.get('https://raw.githubusercontent.com/alekseyst/text_analysis_2025/main/Practical_4/alisa.txt')
alice_text = response.text
alice_text = alice_text.replace('\xa0', ' ') #Заменяем неразрывный пробел обычным, техническая мелочь

In [5]:
#Размечаем книгу — это займёт какое-то время.
#Может оказаться быстрее разбить текст на части, но это не обязательно
#Может оказаться быстрее выполнить токенизацию другим инструментом

alice_annotated = nlp(alice_text)

In [42]:
#Пройдёмся по каждому слову в книге и добавим в табличку

#Создаём заголовок
cols = ['index', 'word', 'lemma', 'pos', 'feats', 'synt']

#Создаём вспомогательный список
words = []

#Добавляем разметку во вспомогательный список
for word_index, word in enumerate(alice_annotated.iter_words()):
    words.append([word_index, word.text, word.lemma, word.upos, word.feats,
              word.deprel])

#Создаём таблицу
alice_by_word = pd.DataFrame(words, columns = cols)

In [7]:
#Таблицу можно дальше записать в файл
alice_by_word.to_csv('alice_corpus_words.tsv', sep = '\t', encoding='utf-8',  index = False)

In [8]:
#А потом считать из файла на компьютере или из интернета
alice_link = 'https://raw.githubusercontent.com/alekseyst/text_analysis_2025/main/Practical_4/alice_corpus_words.tsv'

alice_by_word = pd.read_csv(alice_link, sep='\t')

## Работа с таблицами с помощью Pandas

С полученной таблицей можно сделать очень много всего: извлечь любые данные, аггрегировать, представить в виде, удобном для построения графиков или статистического анализа.

Мы будем это делать с помощью специально библиотеки Pandas. Это очень мощный инструмент, который имеет много встроенных операций, которые можно производить над таблицами, а также даёт возможность применять к ним произвольные функции.

## Базовые действия с таблицей

Pandas предлагает много встроенных функций, которые могут вывести информацию о таблице и её фрагментах.

In [44]:
#Смотрим начало таблицы

alice_by_word.head(15)

,index,word,lemma,pos,feats,synt
0,0,Льюис,Льюис,PROPN,Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing,root
1,1,Кэрролл,Кэрролл,PROPN,Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing,flat:name
2,2,Приключения,приключение,NOUN,Animacy=Inan|Case=Nom|Gender=Neut|Number=Plur,root
3,3,Алисы,Алиса,PROPN,Animacy=Anim|Case=Gen|Gender=Fem|Number=Sing,nmod
4,4,в,в,ADP,None,case
5,5,стране,страна,NOUN,Animacy=Inan|Case=Loc|Gender=Fem|Number=Sing,nmod
6,6,чудес,чудо,NOUN,Animacy=Inan|Case=Gen|Gender=Neut|Number=Plur,nmod
7,7,–,–,PUNCT,None,punct
8,8,Перевод,перевод,NOUN,Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing,root
9,9,Н.,Н.,PROPN,Animacy=Anim|Case=Gen|Gender=Masc|Number=Sing,nmod


In [10]:
#Выводим информацию о таблице

alice_by_word.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40682 entries, 0 to 40681
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   40682 non-null  int64 
 1   word    40682 non-null  object
 2   lemma   40682 non-null  object
 3   pos     40682 non-null  object
 4   feats   23405 non-null  object
 5   synt    40682 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.9+ MB


In [11]:
#Суммируем информацию по численным параметрам таблицы

alice_by_word.describe()

,index
count,40682.000000
mean,20340.500000
std,11744.026162
min,0.000000
25%,10170.250000
50%,20340.500000
75%,30510.750000
max,40681.000000


In [62]:
#Выводим уникальные значения в столбце

alice_by_word['pos'].unique()

array(['PROPN', 'NOUN', 'ADP', 'PUNCT', 'CCONJ', 'X', 'ADJ', 'NUM', 'SYM',
       'VERB', 'ADV', 'PRON', 'DET', 'SCONJ', 'AUX', 'PART', 'INTJ'],
      dtype=object)

Иногда мы хотим работать не со всей таблицей, а только с отдельными её столбцами или строками.

In [63]:
#Столбцы

alice_by_word.loc[:, 'lemma':'feats']

,lemma,pos,feats
0,Льюис,PROPN,Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
1,Кэрролл,PROPN,Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
2,приключение,NOUN,Animacy=Inan|Case=Nom|Gender=Neut|Number=Plur
3,Алиса,PROPN,Animacy=Anim|Case=Gen|Gender=Fem|Number=Sing
4,в,ADP,NaN
...,...,...,...
40677,",",PUNCT,NaN
40678,год,NOUN,Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing
40679,iv,NUM,NumType=Card
40680,),PUNCT,NaN


In [14]:
#Строки

alice_by_word.loc[2:3]

,index,word,lemma,pos,feats,synt
2,2,Приключения,приключение,NOUN,Animacy=Inan|Case=Nom|Gender=Neut|Number=Plur,root
3,3,Алисы,Алиса,PROPN,Animacy=Anim|Case=Gen|Gender=Fem|Number=Sing,nmod


In [15]:
#Строки и столбцы

alice_by_word.loc[2:3, 'lemma':'feats']

,lemma,pos,feats
2,приключение,NOUN,Animacy=Inan|Case=Nom|Gender=Neut|Number=Plur
3,Алиса,PROPN,Animacy=Anim|Case=Gen|Gender=Fem|Number=Sing


## Задание 0

Известно, что глава 4 начинается на токене 6471, а заканчивается на 9098. Выведите эту главу.

In [49]:
#Впишите сюда свой код

chapter_4 = alice_by_word.loc[6471:9098]

## Поиск и логические операции

Поиск по значениям ячеек в Pandas делается с помощью логических операторов, при этом некоторые логические операторы выглядят привычным образом, а некоторые нет, например:

- == — равно
- != — не равно
- & — и
- | — или

Действуют и другие (>, < и т.д.).

Помните, что скобками удобно задать порядок операций!

In [57]:
#Ищем по любой колонке — например, по лемме

alice_by_word[alice_by_word['lemma'] == 'день'].head(15)

,index,word,lemma,pos,feats,synt
73,73,день,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing,obl
213,213,дней,день,NOUN,Animacy=Inan|Case=Gen|Gender=Masc|Number=Plur,nmod
1613,1613,день,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing,obl
2988,2988,день,день,NOUN,Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing,nsubj
9267,9267,день,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing,obl
10711,10711,день,день,NOUN,Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing,obj
10872,10872,день,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing,obl
11891,11891,день,день,NOUN,Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing,conj
11893,11893,днем,день,NOUN,Animacy=Inan|Case=Ins|Gender=Masc|Number=Sing,nmod
12408,12408,днем,день,NOUN,Animacy=Inan|Case=Ins|Gender=Masc|Number=Sing,obl


In [55]:
#Как это работает? Квадратные скобки позволяют выбрать строки таблицы по шаблону
#А в скобки мы пишем тот самый шаблон в виде логического выражения

alice_by_word['lemma'] == 'день'

,lemma
0,False
1,False
2,False
3,False
4,False
...,...
40677,False
40678,False
40679,False
40680,False


In [61]:
#Ищем по больше, чем одному параметру

alice_by_word[(alice_by_word['lemma'] == 'стекло') & (alice_by_word['pos'] == 'NOUN')]

,index,word,lemma,pos,feats,synt
2143,2143,стекло,стекло,NOUN,Animacy=Inan|Case=Acc|Gender=Neut|Number=Sing,obl
7589,7589,стекло,стекло,NOUN,Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing,nsubj
7666,7666,стекло,стекло,NOUN,Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing,nsubj
7813,7813,стекла,стекло,NOUN,Animacy=Inan|Case=Nom|Gender=Neut|Number=Plur,nsubj
33443,33443,стекла,стекло,NOUN,Animacy=Inan|Case=Nom|Gender=Neut|Number=Plur,nsubj:pass


In [20]:
#Ищем любой из списка элементов

alice_by_word[alice_by_word['lemma'].isin(['синий', 'гусеница'])]

,index,word,lemma,pos,feats,synt
9072,9072,синей,синий,ADJ,Case=Ins|Degree=Pos|Gender=Fem|Number=Sing,amod
9073,9073,гусеницей,гусеница,NOUN,Animacy=Inan|Case=Ins|Gender=Fem|Number=Sing,obl
9101,9101,СИНЯЯ,синий,ADJ,Case=Nom|Degree=Pos|Gender=Fem|Number=Sing,amod
9102,9102,ГУСЕНИЦА,гусеница,NOUN,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing,appos
9107,9107,Синяя,синий,PROPN,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing,conj
9145,9145,Синяя,синий,PROPN,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing,nsubj
33481,33481,синий,синий,ADJ,Case=Nom|Degree=Pos|Gender=Masc|Number=Sing,amod
34921,34921,Гусеница,гусеница,NOUN,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing,nsubj


In [24]:
# Можно искать по подстроке, а не по полному совпадению

alice_by_word[(alice_by_word['lemma'] == 'день') &
              (alice_by_word['feats'].str.contains('Number=Plur', na=False))]

,index,word,lemma,pos,feats,synt
213,213,дней,день,NOUN,Animacy=Inan|Case=Gen|Gender=Masc|Number=Plur,nmod
19731,19731,дни,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur,obl
27823,27823,дни,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur,conj
28619,28619,дни,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur,obl
29062,29062,дни,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur,nmod
31621,31621,дни,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur,obl
33220,33220,дни,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur,obl
34708,34708,дни,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur,obl
39654,39654,дни,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur,obl


#### Задание 1

Найдите все пунктуационные знаки в книге.

In [70]:
#Впишите сюда свой код

alice_by_word[alice_by_word['pos'] == 'PUNCT']

,index,word,lemma,pos,feats,synt
7,7,–,–,PUNCT,None,punct
12,12,–,–,PUNCT,None,punct
19,19,",",",",PUNCT,None,punct
27,27,–,–,PUNCT,None,punct
31,31,–,–,PUNCT,None,punct
...,...,...,...,...,...,...
40674,40674,«,"""",PUNCT,None,punct
40676,40676,»,"""",PUNCT,None,punct
40677,40677,",",",",PUNCT,None,punct
40680,40680,),),PUNCT,None,punct


In [73]:
#Создаём частотный список для столбца
#.groupby() группирует таблицу по значением одного из столбцов, собирает вместе
#После этого со сгруппированными значениями можно что-то сделать: посчитать,
#сложить, найти среднее и т. д. Методы .size() и .count() — считают элементы

alice_by_word.groupby('lemma').size().nlargest(15)

,0
lemma,
",",3531
–,2104
.,2083
и,1006
в,828
"""",727
не,664
она,637
!,619


#### Задание 2

Создайте частотный список так, чтобы в нём не было пунктуационных знаков.

In [80]:
#Впишите сюда свой код

alice_no_punct = alice_by_word[alice_by_word['pos'] != 'PUNCT']

alice_no_punct.groupby('lemma').size()

,0
lemma,
#,104
#f6,1
#f8,1
#f9,1
*,30
...,...
ясность,1
ясный,6
ящерица,1


In [83]:
#Более продвинутый уровень — к какому-то из столбцов можно применить функцию
#Например, посчитать количество символов для каждого из текстов, который у нас есть


alice_by_word['len_word'] = alice_by_word['word'].map(len)

#Можем выбрать этот столбец, чтобы посмотреть на него
alice_by_word['len_word'].head(15)

,len_word
0,5
1,7
2,11
3,5
4,1
5,6
6,5
7,1
8,7
9,2


In [86]:
#Можем посмотреть столбец в самой таблице

alice_by_word

,index,word,lemma,pos,feats,synt,len_word
0,0,Льюис,Льюис,PROPN,Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing,root,5
1,1,Кэрролл,Кэрролл,PROPN,Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing,flat:name,7
2,2,Приключения,приключение,NOUN,Animacy=Inan|Case=Nom|Gender=Neut|Number=Plur,root,11
3,3,Алисы,Алиса,PROPN,Animacy=Anim|Case=Gen|Gender=Fem|Number=Sing,nmod,5
4,4,в,в,ADP,None,case,1
...,...,...,...,...,...,...,...
40677,40677,",",",",PUNCT,None,punct,1
40678,40678,гл.,год,NOUN,Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing,conj,3
40679,40679,IV,iv,NUM,NumType=Card,nummod:entity,2
40680,40680,),),PUNCT,None,punct,1


In [89]:
#Численный параметр можно аггрегировать и посчитать для неё необходимую метрику
#Например, среднее:

print(alice_by_word.aggregate('len_word').mean())

3.9708470576667816


In [27]:
#Медиану

print(alice_by_word.aggregate('len_word').median())

3.0


In [28]:
#Дисперсию, и так далее

print(alice_by_word.aggregate('len_word').var())

9.272176805153418


#### Задание 3 (со звёздочкой)

Одна из вещей, которые мы хотели уметь делать в начале курса — находить, в какие связи слово (чаще) вступает с другими словами. Мы дошли до момента, когда можем это сделать. Это можно сделать разными способами — возможно, удобнее всего будет немного изменить разметку корпуса.

Чтобы не тратить время на разметку всей книги, можете использовать её фрагмент, например, приведённый ниже фрагмент.

In [96]:
#Впишите сюда свой код

#Получаем фрагмент книги, который не является автором или названием
shorter_alice_text = '\n'.join(alice_text.split('\r\n\r\n')[18:23])

In [108]:
shorter_annotated = nlp(shorter_alice_text)

In [127]:
#Создаём заголовок
cols = ['index', 'word', 'lemma', 'pos', 'feats', 'synt', 'head']

#Создаём вспомогательный список
words = []

#Добавляем разметку во вспомогательный список

for sentence_index, sentence in enumerate(shorter_annotated.sentences):

  for word_index, word in enumerate(sentence.words):

    head_index = shorter_annotated.sentences[sentence_index].words[word_index].head
    head = shorter_annotated.sentences[sentence_index].words[head_index - 1]

    words.append([word_index, word.text, word.lemma, word.upos, word.feats,
              word.deprel, head.lemma])

#Создаём таблицу
shorter_by_word = pd.DataFrame(words, columns = cols)

In [126]:
shorter_by_word[(shorter_by_word['lemma'] == 'она') & (shorter_by_word['head'] == 'сидеть')]

,index,word,lemma,pos,feats,synt,head
0,0,Она,она,PRON,Case=Nom|Gender=Fem|Number=Sing|Person=3|PronT...,nsubj,сидеть


## Более сложная структура корпуса

Конечно, мы можем работать с корпусом, который состоит больше, чем из одной книги. Его структура может быть разной в зависимости от ваших данных.

Может быть нужно хранить мета-информацию о текстах, разметка по словам, гипотетически может быть разметка на уровне предложений. Всю эту информацию имеет смысл не смешивать, а хранить в отдельных табличках (при этом мы фактически создаём реляционную базу данных).

Чтобы понимать, как таблички соотносятся друг с другом, нужны индексы.

Создадим корпус, который содержит новости, скачанные со страницы Панорамы.

Будем работать с готовой таблицей, в которой содержится метаинформация (при небольшом количестве элементов она может быть создана вручную, при большом — автоматически), и с ещё одной таблицей, в которой будет разметка для каждого слова.

In [30]:
#Считываем из файла табличку с метаинформацией о каждом файле (включая текст новости)

panorama_corpus_link = 'https://raw.githubusercontent.com/alekseyst/text_analysis_2025/main/Practical_4/panorama_corpus.tsv'

panorama_corpus = pd.read_csv(panorama_corpus_link, sep='\t')

In [31]:
panorama_corpus.head()

,index,date,sphere,title,text,link
0,0,4-3-2023,Общество,Гражданам разрешили добывать нефть и газ на св...,Государственная дума приняла в третьем чтении ...,https://panorama.pub/news/grazdanam-razresili-...
1,1,4-3-2023,Политика,Китай подал заявку на вступление в Союзное гос...,Заведующий Канцелярией Комиссии ЦК КПК по инос...,https://panorama.pub/news/kitaj-podal-zaavku-n...
2,2,4-3-2023,Политика,Деколонизаторки из Франции потребовали вернуть...,Активисты движения «Смерть колониализму» из Па...,https://panorama.pub/news/dekolonizatorki-iz-f...
3,3,4-3-2023,Политика,Победа России: Госдума пересмотрела результаты...,Государственная дума приняла постановление «Об...,https://panorama.pub/news/gosduma-peresmotrela...
4,4,3-3-2023,Общество,Бездетных россиян будут ежегодно штрафовать,Государственная дума приняла в первом чтении з...,https://panorama.pub/news/bezdetnyh-rossian-pr...


In [32]:
#Возьмём для скорости поменьше новостей

panorama_corpus = panorama_corpus.iloc[:50]

In [33]:
#Пройдёмся по каждому слову каждой из новостей и добавим в новую табличку
cols = ['index', 'text_index', 'word', 'lemma', 'pos', 'feats',
        'synt',  'head']
words = []

for text_index, row in enumerate(panorama_corpus['text']):
  news_nlp = nlp(row)
  for word_index, word in enumerate(news_nlp.iter_words()):
    words.append([word_index, text_index, word.text, word.lemma, word.upos,
                  word.feats, word.deprel, word.head])

panorama_by_word = pd.DataFrame(words, columns = cols)

In [34]:
panorama_by_word.head()

,index,text_index,word,lemma,pos,feats,synt,head
0,0,0,Государственная,государственный,ADJ,Case=Nom|Degree=Pos|Gender=Fem|Number=Sing,amod,2
1,1,0,дума,дума,NOUN,Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing,nsubj,3
2,2,0,приняла,принять,VERB,Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Te...,root,0
3,3,0,в,в,ADP,None,case,6
4,4,0,третьем,третий,ADJ,Case=Loc|Degree=Pos|Gender=Neut|Number=Sing,amod,6


In [35]:
panorama_by_word.to_csv('panorama_corpus_words.tsv', sep = '\t', encoding='utf-8',  index = False)

In [36]:
panorama_by_word_link = 'https://raw.githubusercontent.com/alekseyst/text_analysis_2025/main/Practical_4/panorama_corpus_words.tsv'

panorama_by_word = pd.read_csv(panorama_by_word_link, sep='\t')

In [37]:
#Если нужно использовать информацию сразу из двух таблиц, их можно объединить
#Только не целиком — иначе получится слишком огромная таблица, которая потратит
#Все наши ресурсы!

#Получим таблицу, которая содержит дату создания примера для каждого слова.
#Объединять будем по индексу текста

panorama_by_word_merged = panorama_by_word.join(
    panorama_corpus.loc[:, ['index', 'date']].set_index('index'),
    on='text_index')

panorama_by_word_merged

,index,text_index,word,lemma,pos,feats,synt,head,date
0,0,0,Государственная,государственный,ADJ,Case=Nom|Degree=Pos|Gender=Fem|Number=Sing,amod,2,4-3-2023
1,1,0,дума,дума,NOUN,Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing,nsubj,3,4-3-2023
2,2,0,приняла,принять,VERB,Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Te...,root,0,4-3-2023
3,3,0,в,в,ADP,NaN,case,6,4-3-2023
4,4,0,третьем,третий,ADJ,Case=Loc|Degree=Pos|Gender=Neut|Number=Sing,amod,6,4-3-2023
...,...,...,...,...,...,...,...,...,...
9459,145,49,организацией,организация,NOUN,Animacy=Inan|Case=Ins|Gender=Fem|Number=Sing,obl,4,24-2-2023
9460,146,49,",",",",PUNCT,NaN,punct,4,24-2-2023
9461,147,49,ещё,еще,ADV,Degree=Pos,advmod,8,24-2-2023
9462,148,49,оценивается,оценивать,VERB,Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense...,root,0,24-2-2023
